In [1]:
import os
import torch
import numpy as np
import math
import re
import matplotlib.pyplot as plt
from glob import glob
from natsort import natsorted
from tqdm import tqdm
import scipy.ndimage

/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <

In [2]:
all_image_paths = natsorted(glob('/mnt/home/alu10/ceph/codebases/registration/registered_sequences/registered_sequence_*/synced_images/synced_*.npy'))
all_mask_paths = natsorted(glob('/mnt/home/alu10/ceph/codebases/registration/registered_sequences/registered_sequence_*/synced_masks/synced_*.npy'))

In [3]:
def extract_sequence_and_timepoint(path):
    # Extract sequence name from the directory
    sequence_match = re.search(r'registered_sequence_([^/]+)', path)
    if not sequence_match:
        raise ValueError(f"Could not find sequence in path: {path}")
    sequence_name = sequence_match.group(1)

    # Extract timepoint from the filename
    filename = os.path.basename(path)
    timepoint_match = re.search(r'_(\d+)\.npy$', filename)
    if not timepoint_match:
        raise ValueError(f"Could not find timepoint in filename: {filename}")
    timepoint = int(timepoint_match.group(1))

    timepoint_name = filename.split('synced_')[-1].split('.npy')[0]

    return sequence_name, timepoint_name, timepoint, 

In [4]:
def crop_with_overlap(array, crop_shape, overlap):
    z, y, x = array.shape
    crop_z, crop_y, crop_x = crop_shape
    overlap_z, overlap_y, overlap_x = overlap

    assert crop_z <= z and crop_y <= y and crop_x <= x
    assert overlap_z <= crop_z and overlap_y <= crop_y and overlap_x <= crop_x

    crops = []

    for z_start in range(0, z, crop_z - overlap_z):
        for y_start in range(0, y, crop_y - overlap_y):
            for x_start in range(0, x, crop_x - overlap_x):
                z_end = min(z_start + crop_z, z)
                y_end = min(y_start + crop_y, y)
                x_end = min(x_start + crop_x, x)

                crop = array[z_end - crop_z:z_end, y_end - crop_y:y_end, x_end - crop_x:x_end]
                assert crop.shape == crop_shape
                crops.append(crop)

    return crops

crop_shape = (256, 256, 256)
overlap = (0, 0, 0)

all_image_crops = []
all_mask_crops = []

for image_path, mask_path in zip(all_image_paths, all_mask_paths):
    sequence_name, timepoint_name, timepoint = extract_sequence_and_timepoint(image_path)
        
    try:
        iso_image = np.load(image_path)
        iso_mask = np.load(mask_path)
    except Exception as e:
        print(f"Skipping {image_path} due to load error: {e}")
        continue

    image_crops = crop_with_overlap(iso_image, crop_shape, overlap)
    mask_crops = crop_with_overlap(iso_mask, crop_shape, overlap)

    all_image_crops.extend(image_crops)
    all_mask_crops.extend(mask_crops)

    print(f"{timepoint_name} - Number of crops: {len(image_crops)}")

    for i, (image_crop, mask_crop) in enumerate(zip(image_crops, mask_crops)):
        cropped_image_dir = f'/mnt/ceph/users/alu10/datasets/256x256x256_Registered_Iso_Blastospim1-Trilinear_Image/{sequence_name}/{timepoint_name}/images'
        cropped_mask_dir = f'/mnt/ceph/users/alu10/datasets/256x256x256_Registered_Iso_Blastospim1-Trilinear_Image/{sequence_name}/{timepoint_name}/masks'

        os.makedirs(cropped_image_dir, exist_ok=True)
        os.makedirs(cropped_mask_dir, exist_ok=True)

        cropped_image_path = os.path.join(cropped_image_dir, f'{timepoint_name}_image_{i+1:04d}.npy')
        cropped_mask_path = os.path.join(cropped_mask_dir, f'{timepoint_name}_masks_{i+1:04d}.npy')

        # Skip if both cropped files already exist
        if os.path.exists(cropped_image_path) and os.path.exists(cropped_mask_path):
            print(f"Skipping existing crops: {cropped_image_path}, {cropped_mask_path}")

        np.save(cropped_image_path, image_crop)
        np.save(cropped_mask_path, mask_crop)


H4_020 - Number of crops: 8
